In [ ]:
import os
import shutil
import pandas as pd

# Directorio de imágenes preprocesadas
IMAGE_DIR = "../04_Extra/ID/IMAGES_PREPROC"

# Cargar dataset con información de los sets y sus años de retiro
df_lego = pd.read_csv("../04_Extra/ID/df_lego_final_retirados.csv")

# Filtrar sets con YearExit > 2004
sets_validos = df_lego[df_lego["YearExit"] <= 2004]["set_id"].astype(str).tolist()

# 🔹 1️⃣ Eliminar carpetas con una sola imagen
for set_folder in os.listdir(IMAGE_DIR):
    set_path = os.path.join(IMAGE_DIR, set_folder)
    
    if os.path.isdir(set_path):
        num_images = len(os.listdir(set_path))  # Contar imágenes en la carpeta
        
        # Si solo hay 1 imagen o el set no está en la lista de sets válidos, eliminar carpeta
        if num_images <= 1 or set_folder not in sets_validos:
            shutil.rmtree(set_path)  # Eliminar carpeta y contenido
            print(f"❌ Eliminado: {set_folder} (Imágenes: {num_images}, YearExit > 2004)")
